## RTP Instability
Reduction-to-pole (RTP) is one of the most widely used data processing techniques in potential field geophysics. The typical way of performing RTP is in Fourier domain. When it comes to teaching, however, it is often hard to explain to students why the standard RTP method would fail at low magnetic latitude without going into the details of some complicated mathematics. This script is to help students in GEOL 7330 at UH to understand the instabilities of RTP operator in wavenumber doman. This is particularly useful for those who do not want to get into the mathematical details, or who do not want to do the coding work themselves. This notebook allows students to be active and play with several adjustable parameters to really understand how RTP operator works. <br>
<br>
Author: Jiajia Sun, 03/31/2020 at University of Houston.

### First step is to import some packages that we will need to do the mathematical calculation.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cmath
import ipywidgets as widgets
from IPython.display import display

In [7]:
# to get rid of the warning message when dividing by 0 (i.e, when kx=ky=0)
import warnings
warnings.filterwarnings('ignore')

### Create a function that convert dip and azimuth to a three-component vector in Cartesian coordinate system.

In [8]:
def degree2_cartesian(inc,dec):
    inc_radian = np.radians(inc)
    dec_radian = np.radians(dec)
    return np.array([np.cos(inc_radian)*np.cos(dec_radian),np.cos(inc_radian)*np.sin(dec_radian),np.sin(inc_radian)])

### Create an interactive widget to allow users to change the input values and visualize the results

In [10]:
@widgets.interact_manual(inc_induce=(-90,90),dec_induce=(0,360),
                        inc_magnetiz=(-90,90),dec_magnetiz=(0,360),
                        color=['jet','viridis','RdYlBu_r'])
def plot(inc_induce=60,dec_induce=30,inc_magnetiz=60,dec_magnetiz=30,grid=True,color='jet'): 
    kx = np.arange(-100,101)
    ky = np.arange(-100,101)
    f = degree2_cartesian(inc_induce,dec_induce) # inducing field
    m = degree2_cartesian(inc_magnetiz,dec_magnetiz) # magnetization direction
    RTP_amp = np.zeros((np.size(ky),np.size(kx)))
    RTP_phs = np.zeros((np.size(ky),np.size(kx)))
    for icount_x in np.arange(np.size(kx)):
        for icount_y in np.arange(np.size(ky)):
            k_abs = np.sqrt(kx[icount_x]**2+ky[icount_y]**2)
            R = np.array([kx[icount_x]*1j,ky[icount_y]*1j,k_abs])
            denominator = R.dot(f)*R.dot(m)
            RTP_Operator = k_abs**2/denominator
            RTP_amp[icount_y,icount_x] = abs(RTP_Operator)
            RTP_phs[icount_y,icount_x] = cmath.phase(RTP_Operator)
            
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(1,2,1)
    plt.imshow(RTP_amp,cmap=color)
    ax.set_title('Amplitude spectrum')
    ax.set_aspect('equal')
    ax.grid(grid)
    pos = ax.get_position()
    cax = fig.add_axes([pos.x0,pos.y0-0.08,pos.width*1,pos.height*0.05])
    plt.colorbar(cax=cax,orientation='horizontal')

    ax2 = plt.subplot(1,2,2)
    plt.imshow(RTP_phs,cmap=color)
    ax2.set_title('Phase spectrum')
    ax2.set_aspect('equal')
    ax2.grid(grid)
    pos2 = ax2.get_position()
    cax2 = fig.add_axes([pos2.x0,pos2.y0-0.08,pos2.width*1,pos2.height*0.05])
    plt.colorbar(cax=cax2,orientation='horizontal')
    plt.show()

interactive(children=(IntSlider(value=60, description='inc_induce', max=90, min=-90), IntSlider(value=30, desc…